In [2]:
import pandas as pd

file_path = '../data/cleaned_data.tsv'
df = pd.read_csv(file_path, sep='\t')

df.head()


,Title,Author,Description,Content,Published At,URL,Coding
0,"Joe Rogan endorsed Donald Trump, saying Elon M...",Erin Snodgrass,"Joe Rogan, who interviewed Donald Trump on his...",Joe Rogan endorsed Donald Trump ahead of the e...,2024-11-05T02:52:54Z,https://www.businessinsider.com/joe-rogan-endo...,NaN
1,Trump praises Elon Musk in election night spee...,Brent D. Griffiths,"Donald Trump called Tesla CEO Elon Musk ""an am...",Former President Donald Trump basked in the ne...,2024-11-06T08:05:03Z,https://www.businessinsider.com/trump-praises-...,NaN
2,Mark Cuban praises Trump's 'great personality'...,Lloyd Lee,"Mark Cuban told Business Insider that ""we all ...","""I truly don't hate him at all. I just think h...",2024-10-26T02:48:21Z,https://www.businessinsider.com/mark-cuban-don...,NaN
3,What the Trump biopic 'The Apprentice' got rig...,Eve Crosbie,"The new Donald Trump biopic, which opened in U...",Sebastian Stan transforms into Donald Trump in...,2024-10-19T10:44:15Z,https://www.businessinsider.com/the-apprentice...,NaN
4,Ivanka Trump has been absent from the 2024 cam...,Talia Lakritz,Ivanka Trump stepped back from politics when D...,Ivanka Trump is no longer active in politics.J...,2024-10-24T13:14:02Z,https://www.businessinsider.com/ivanka-trump-d...,NaN


In [4]:
df_sample = df.sample(n=200, random_state=1)
df_sample = df_sample[['Title', 'Content', 'Coding']]
df_sample.to_csv('../data/sample_data_for_oc.tsv', sep='\t', index=False)



In [17]:
#For the sample annotated data, count the coding column and how many of each coding there are
labeled_sample = pd.read_csv('../data/labeled_sample_data_for_oc.tsv', sep='\t')
count = labeled_sample['Coding'].value_counts()
print(count)

Coding
Unlabeled                                    68
Candidate Campaigning & Strategy             21
Election Coverage & Polling Results          20
Miscellaneous Campaign Events & Anecdotes    16
Candidate Endorsements & Influencers         16
Economic and Market Impact                   15
Legal & Controversy Coverage                 15
Social & Cultural Commentary                  8
International Relations & Foreign Policy      6
Media & Public Opinion Analysis               5
Policy Proposals & Political Stances          5
Personal & Character Assessments              5
Name: count, dtype: int64


In [4]:
data = pd.read_csv('../data/labeled_sample_data_for_oc.tsv', sep='\t')
print(data['Coding'].value_counts())

Coding
Election Coverage & Polling Results          28
Candidate Campaigning & Strategy             28
Economic and Market Impact                   26
Legal & Controversy Coverage                 23
Miscellaneous Campaign Events & Anecdotes    20
Candidate Endorsements & Influencers         20
Social & Cultural Commentary                 12
Policy Proposals & Political Stances         11
Media & Public Opinion Analysis              10
Personal & Character Assessments             10
International Relations & Foreign Policy      9
Only Name Mention(Other)                      3
Name: count, dtype: int64


In [ ]:
def check_for_duplicates(df1, df2):
    df2_set = set(df2.apply(tuple, axis=1))
    
    unique_df1 = df1[~df1.apply(tuple, axis=1).isin(df2_set)]
    
    return unique_df1


df1 = pd.read_csv('../data/cleaned_data.tsv', sep='\t')
df2 = pd.read_csv('../data/sample_data_for_oc.tsv', sep='\t')
df = check_for_duplicates(df1, df2)
df.to_csv('../data/rest_of_non_annotated.tsv', sep='\t')


In [7]:
def process_and_merge(df1, df2, df3):
    for index, row in df3.iterrows():
        matching_rows = df2[df2['Title'] == row['Title']]
        if not matching_rows.empty:
            df2.loc[matching_rows.index, 'Coding'] = row['Coding']
    merged_df = pd.concat([df1, df2], ignore_index=True)
    
    return merged_df

df1 = pd.read_csv('../data/final/refined_annotated_articles.tsv', sep='\t')
df2 = pd.read_csv('../data/raw/cleaned_data.tsv', sep='\t')
df3 = pd.read_csv('../data/open_coding/labeled_sample_data_for_oc.tsv', sep='\t')
df = process_and_merge(df1, df2, df3)
df.to_csv('../data/final/merged_annotated_articles.tsv', sep='\t', index=False)


C:\Users\ACER\AppData\Local\Temp\ipykernel_1704\2488887615.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Social & Cultural Commentary' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[matching_rows.index, 'Coding'] = row['Coding']


In [14]:
df = pd.read_csv('../data/final/merged_annotated_articles.tsv', sep='\t')
df['Title'].value_counts().head(10)

df.drop_duplicates(subset='Title', keep='first', inplace=True)
df.to_csv('../data/final/merged_annotated_articles.tsv', sep='\t', index=False)


In [18]:
#Given a df, remove the row if any record is empty for that row, also keep only "Title" and "Description" columns
def remove_empty_rows(df):
    df = df.dropna()
    
    return df

df = pd.read_csv('../data/final/merged_annotated_articles.tsv', sep='\t')
df = remove_empty_rows(df)
df.to_csv('../data/final/merged_annotated_articles.tsv', sep='\t', index=False)

In [16]:
#count the number of lines of a txt file 
def count_lines(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file)
    
file_path = 'annotations.txt'
print(count_lines(file_path))


212
